In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, MultiSelect, Button, ColumnDataSource, DateRangeSlider
from bokeh.models import RangeSlider, CheckboxButtonGroup, MultiChoice, TextInput
from bokeh.events import ButtonClick, MenuItemClick
from bokeh.layouts import row, column
output_notebook()
ROOT.gROOT.LoadMacro("../RootMacros/runCCDBItemList.C")
ROOT.gROOT.LoadMacro("../RootMacros/runCCDBDownloadTarget.C")

Welcome to JupyROOT 6.24/06


Loading BokehJS ...

0

### Get the list of objects on the CCDB
First choose from where you want to get the data.
Options:
* TestCCDB
* QCDB
* LocalDB

In [20]:
target = "TestCCDB"
path = "qc_mc/TPC/MO/Tracks"


The /Data/UserFiles/CCDB.csv file should be generated in advance so we know what files are on the CCDB. If you already have this, no need to run the next cell which calls the runCCDBItemList.C macro.

In [21]:
ROOT.runCCDBItemList(target,path)

[INFO] Is alien token present?: 0
Getting list
Vectorizing
Sorting
Writing to file


### Read the file list
Here we import the file list we created into python.

In [22]:
CCDB = pd.read_csv("../../Data/UserFiles/localDB/"+target+"_list.csv",header=0, sep = r',', skipinitialspace = True)
files = list(set(CCDB.Name))
paths = list(set(CCDB.Path))
time_stamps = list(set(CCDB.TimeStamp))
types = list(set(CCDB.Type))
tasks = list(set(CCDB.Task))
sizes = list(set(CCDB.Size))
runNumbers = list(set(CCDB.RunNumber))

### Dashboard to select files to download
Click the download button after selecting the wanted task, object, file type and time range

In [23]:
files_data = CCDB.copy(deep=True)

selected_files = files_data.copy(deep=True)
shown_files = files_data.copy(deep=True)

selected_data = []
selected_types = []
selected_tasks = []
time_range = [files_data['TimeStamp'].min(),files_data['TimeStamp'].max()]
#time_range = [1644415200000,1644419264263] # if you want to hard select the time range
runNum_range = [files_data['RunNumber'].min(),files_data['RunNumber'].max()]

In [24]:
def file_browser(doc):
    def file_size_str(table):
        size = table["Size"].sum()
        if size < 1E3:
            string = "{} Bytes".format(size)
        elif size < 1E6:
            string = "{:.2f} kB".format(size*(1E-3))
        elif size < 1E9:
            string = "{:.2f} MB".format(size*(1E-6))
        else:
            string = "{:.2f} GB".format(size*(1E-9))
        return string
    
    def download_selected(selected_files):
        ROOT.runCCDBDownloadTarget(list(selected_files["ID"]),target,"../../Data/UserFiles/myFile.root",)
        
    def update_button_label():
        global selected_files
        label = "Download {} files ({})".format(str(len(selected_files.index)),file_size_str(selected_files))
        button.label = label
        
    def update_file_select_list():
        global selected_files, time_range, selected_types, selected_tasks, files_data, shown_files
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        shown_files = files_data[type_condition & time_condition & task_condition]
        file_select.options = list(set(shown_files['Name']))
        
    def update_time_range():
        global shown_files
        if not shown_files.empty:
            date_range.value=[shown_files['TimeStamp'].min(),shown_files['TimeStamp'].max()]
            date_range.start=shown_files['TimeStamp'].min()
            date_range.end=shown_files['TimeStamp'].max()
    
    def update_run_range():
        global shown_files
        if not shown_files.empty:
            run_range.value=[shown_files['RunNumber'].min(),shown_files['RunNumber'].max()]
            run_range.start=shown_files['RunNumber'].min()
            run_range.end=shown_files['RunNumber'].max()
    
    def update_type_list():
        global selected_tasks, time_range, files_data
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        existing_types = files_data[time_condition & task_condition]
        type_select.options = list(set(existing_types['Type']))
        
    def update_selection():
        global selected_data, selected_files, time_range, selected_types, selected_tasks, files_data, runNum_range
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        name_condition = files_data['Name'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        run_condition = ((files_data['RunNumber'] >= runNum_range[0]) & (files_data['RunNumber'] <= runNum_range[1]))
        selected_files = files_data[name_condition & time_condition & type_condition & task_condition & run_condition]
    
    def select_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, selected_tasks
        selected_data = file_select.value
        update_selection()
        update_button_label()
        
    def date_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        time_range = date_range.value
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def run_callback(attr,old,new):
        global selected_data, selected_files, files_data, runNum_range, selected_types, shown_files, selected_tasks
        runNum_range = run_range.value
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def type_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_types = type_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        update_run_range()
        
    def task_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_tasks = task_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        update_type_list()
        update_run_range()
        
    def timestamp_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        timestamp = int(timestamp_text.value)
        time_range = [timestamp-400000,timestamp+400000]
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def button_callback(event):
        global selected_data, selected_files, files_data, time_range, selected_types
        display(selected_files)
        print("Downloading {} files".format(str(len(selected_files.index))))
        download_selected(selected_files)

    type_select = MultiChoice(title='Object type',options=types,value=[])
    type_select.on_change("value",type_callback)
    
    task_select = MultiChoice(title='QC task',options=tasks,value=[])
    task_select.on_change("value",task_callback)
        
    date_range = DateRangeSlider(title='Time range',value=[files_data['TimeStamp'].min(),files_data['TimeStamp'].max()],
                                    start=files_data['TimeStamp'].min(), end=files_data['TimeStamp'].max())
    date_range.on_change("value_throttled",date_callback)    
    
    run_range = RangeSlider(title='Run Number',value=[files_data['RunNumber'].min(),files_data['RunNumber'].max()],
                                    start=files_data['RunNumber'].min()-1, end=files_data['RunNumber'].max())
    run_range.on_change("value_throttled",run_callback)
    
    timestamp_text = TextInput(value=str(runNum_range[0]), title="Timestamp:", width=150)
    timestamp_text.on_change("value",timestamp_callback)
    
    file_select = MultiSelect(title="Files", value=[], options=files, height=300)
    file_select.on_change("value", select_callback)

    button = Button(label="Download all ({})".format(file_size_str(files_data)), button_type="success") 
    button.on_event(ButtonClick, button_callback)
    
    doc.add_root(row(column(task_select,type_select,date_range,run_range,timestamp_text,button),file_select))
show(file_browser)

,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
141,142,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1650724483953,TH1F,Tracks,5120,302014
287,288,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1650724485214,TH2F,Tracks,302080,302014
341,342,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1650724484503,TH2F,Tracks,97280,302014
352,353,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1650724483556,TH1F,Tracks,4096,302014
680,681,qc_mc/TPC/MO/Tracks/,hPt,1650724483728,TH1F,Tracks,5120,302014
1636,1637,qc_mc/TPC/MO/Tracks/,h2DNClustersEta,1650724484333,TH2F,Tracks,69632,302014
2030,2031,qc_mc/TPC/MO/Tracks/,hPtNeg,1650724484082,TH1F,Tracks,5120,302014
2150,2151,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1650724483910,TH1F,Tracks,5120,302014
2323,2324,qc_mc/TPC/MO/Tracks/,hEtaNeg,1650724483828,TH1F,Tracks,4096,302014
2613,2614,qc_mc/TPC/MO/Tracks/,hPhiAside,1650724483641,TH1F,Tracks,5120,302014


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
91,92,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1650846755106,TH1F,Tracks,5120,302015
305,306,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1650846755238,TH2F,Tracks,97280,302015
388,389,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1650846755054,TH1F,Tracks,4096,302015
428,429,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1650846755401,TH2F,Tracks,304128,302015
622,623,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1650846755527,TH2F,Tracks,282624,302015
1021,1022,qc_mc/TPC/MO/Tracks/,hPt,1650846755075,TH1F,Tracks,5120,302015
1715,1716,qc_mc/TPC/MO/Tracks/,h2DNClustersEta,1650846755195,TH2F,Tracks,70656,302015
2105,2106,qc_mc/TPC/MO/Tracks/,hPtNeg,1650846755141,TH1F,Tracks,5120,302015
2227,2228,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1650846755099,TH1F,Tracks,5120,302015
2401,2402,qc_mc/TPC/MO/Tracks/,hEtaNeg,1650846755087,TH1F,Tracks,4096,302015


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
375,376,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1650972647833,TH2F,Tracks,304128,302016
406,407,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1650972647463,TH2F,Tracks,97280,302016
475,476,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1650972647290,TH1F,Tracks,5120,302016
573,574,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1650972647973,TH2F,Tracks,282624,302016
1086,1087,qc_mc/TPC/MO/Tracks/,hPt,1650972647182,TH1F,Tracks,5120,302016
1651,1652,qc_mc/TPC/MO/Tracks/,h2DNClustersEta,1650972647415,TH2F,Tracks,70656,302016
1908,1909,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1650972647109,TH1F,Tracks,4096,302016
2058,2059,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1650972647255,TH1F,Tracks,5120,302016
2087,2088,qc_mc/TPC/MO/Tracks/,hPtNeg,1650972647336,TH1F,Tracks,5120,302016
2421,2422,qc_mc/TPC/MO/Tracks/,hEtaNeg,1650972647196,TH1F,Tracks,4096,302016


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': '1202'}, 'attr': 'value', 'new': ''}], 'references': []} 
 error: ValueError("invalid literal for int() with base 10: ''",)
Traceback (most recent call last):
  File "/home/berki/alice/sw/ubuntu2004_x86-64/Python/v3.6.10-2/lib/python/site-packages/bokeh/server/protocol_handler.py", line 90, in handle
    work = await handler(message, connection)
  File "/home/berki/alice/sw/ubuntu2004_x86-64/Python/v3.6.10-2/lib/python/site-packages/bokeh/server/session.py", line 67, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "/home/berki/alice/sw/ubuntu2004_x86-64/Python/v3.6.10-2/lib/python/site-packages/bokeh/server/session.py", line 261, in _handle_patch
    message.apply_to_document(self.document, self)
  File "/home/berki/alice/sw/ubuntu2004_x86-64/Python/v3.6.10-2/lib/python/site-packages/bokeh/protocol/message

,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
346,347,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651125379894,TH1F,Tracks,5120,302018
812,813,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651125386568,TH2F,Tracks,35840,302018
937,938,qc_mc/TPC/MO/Tracks/,hPt,1651125377687,TH1F,Tracks,5120,302018
1038,1039,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651125400793,TH2F,Tracks,282624,302018
1305,1306,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651125379105,TH1F,Tracks,5120,302018
1403,1404,qc_mc/TPC/MO/Tracks/,hPtNeg,1651125380870,TH1F,Tracks,5120,302018
1524,1525,qc_mc/TPC/MO/Tracks/,h2DNClustersEta,1651125382882,TH2F,Tracks,70656,302018
1755,1756,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651125396122,TH2F,Tracks,305152,302018
1830,1831,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651125376400,TH1F,Tracks,4096,302018
1996,1997,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651125385136,TH2F,Tracks,97280,302018


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
193,194,qc_mc/TPC/MO/Tracks/,hPt,1651186531318,TH1F,Tracks,5120,302019
350,351,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651186531341,TH1F,Tracks,5120,302019
642,643,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651186531451,TH2F,Tracks,34816,302019
1111,1112,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651186531691,TH2F,Tracks,283648,302019
1312,1313,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651186531336,TH1F,Tracks,5120,302019
1370,1371,qc_mc/TPC/MO/Tracks/,hPtNeg,1651186531355,TH1F,Tracks,5120,302019
1734,1735,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651186531589,TH2F,Tracks,305152,302019
1768,1769,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651186531423,TH2F,Tracks,97280,302019
2201,2202,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651186531295,TH1F,Tracks,4096,302019
2699,2700,qc_mc/TPC/MO/Tracks/,h2DNClustersEtaBeforeCuts,1651186531550,TH2F,Tracks,169984,302019


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
193,194,qc_mc/TPC/MO/Tracks/,hPt,1651186531318,TH1F,Tracks,5120,302019
350,351,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651186531341,TH1F,Tracks,5120,302019
642,643,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651186531451,TH2F,Tracks,34816,302019
1111,1112,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651186531691,TH2F,Tracks,283648,302019
1312,1313,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651186531336,TH1F,Tracks,5120,302019
1370,1371,qc_mc/TPC/MO/Tracks/,hPtNeg,1651186531355,TH1F,Tracks,5120,302019
1734,1735,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651186531589,TH2F,Tracks,305152,302019
1768,1769,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651186531423,TH2F,Tracks,97280,302019
2201,2202,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651186531295,TH1F,Tracks,4096,302019
2699,2700,qc_mc/TPC/MO/Tracks/,h2DNClustersEtaBeforeCuts,1651186531550,TH2F,Tracks,169984,302019


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
521,522,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651090318093,TH1F,Tracks,5120,302020
802,803,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651090318978,TH2F,Tracks,34816,302020
1002,1003,qc_mc/TPC/MO/Tracks/,hPt,1651090317536,TH1F,Tracks,5120,302020
1071,1072,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651090320351,TH2F,Tracks,282624,302020
1314,1315,qc_mc/TPC/MO/Tracks/,hPtNeg,1651090318281,TH1F,Tracks,5120,302020
1329,1330,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651090317996,TH1F,Tracks,5120,302020
1515,1516,qc_mc/TPC/MO/Tracks/,h2DNClustersEta,1651090318644,TH2F,Tracks,70656,302020
1728,1729,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651090317315,TH1F,Tracks,4096,302020
1920,1921,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651090318877,TH2F,Tracks,97280,302020
2003,2004,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651090319805,TH2F,Tracks,304128,302020


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
180,181,qc_mc/TPC/MO/Tracks/,hPt,1651212785686,TH1F,Tracks,5120,302022
370,371,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651212785710,TH1F,Tracks,5120,302022
630,631,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651212785816,TH2F,Tracks,34816,302022
1145,1146,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651212786038,TH2F,Tracks,281600,302022
1394,1395,qc_mc/TPC/MO/Tracks/,hPtNeg,1651212785729,TH1F,Tracks,5120,302022
1410,1411,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651212785705,TH1F,Tracks,5120,302022
1729,1730,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651212785945,TH2F,Tracks,303104,302022
1759,1760,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651212785796,TH2F,Tracks,97280,302022
2185,2186,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651212785667,TH1F,Tracks,4096,302022
2694,2695,qc_mc/TPC/MO/Tracks/,h2DNClustersEtaBeforeCuts,1651212785916,TH2F,Tracks,169984,302022


,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
210,211,qc_mc/TPC/MO/Tracks/,hPt,1651354036873,TH1F,Tracks,5120,302023
651,652,qc_mc/TPC/MO/Tracks/,h2DNClustersPt,1651354036983,TH2F,Tracks,34816,302023
767,768,qc_mc/TPC/MO/Tracks/,h2DNClustersPtBeforeCuts,1651354037101,TH2F,Tracks,49152,302023
1091,1092,qc_mc/TPC/MO/Tracks/,h2DQOverPtPhiCside,1651354037235,TH2F,Tracks,281600,302023
1187,1188,qc_mc/TPC/MO/Tracks/,hPtNeg,1651354036912,TH1F,Tracks,5120,302023
1379,1380,qc_mc/TPC/MO/Tracks/,hPhiAsideNeg,1651354036892,TH1F,Tracks,5120,302023
1751,1752,qc_mc/TPC/MO/Tracks/,h2DNClustersPhiCside,1651354036969,TH2F,Tracks,97280,302023
1844,1845,qc_mc/TPC/MO/Tracks/,h2DEtaPhiBeforeCuts,1651354037110,TH2F,Tracks,302080,302023
1903,1904,qc_mc/TPC/MO/Tracks/,hPhiAsidePos,1651354036896,TH1F,Tracks,5120,302023
2165,2166,qc_mc/TPC/MO/Tracks/,hNClustersAfterCuts,1651354036848,TH1F,Tracks,4096,302023
